In [4]:
import os
import csv

In [17]:
# read the first row from the columns "DetectorID" and "Location" in a CSV file
# for making the unique data from two directories - which is all sensor file list in 2018 and 2019

import os
import csv

def read_first_row(file_path):
    """
    Read the first row from the columns "DetectorID" and "Location" in a CSV file.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        tuple: A tuple containing the "DetectorID" and "Location" from the first row.
    """
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        first_row = next(reader)
        detector_id = first_row.get('DetectorID', '')
        location = first_row.get('Location', '')
    return detector_id, location

def get_unique_data(directory1, directory2):
    unique_data = set()
    
    for directory in [directory1, directory2]:
        for filename in os.listdir(directory):
            if filename.endswith('.csv'):
                file_path = os.path.join(directory, filename)
                detector_id, location = read_first_row(file_path)
                if detector_id and location:  # Ensure both values are non-empty
                    unique_data.add((detector_id, location))
    
    return unique_data

def write_to_csv(data, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['DetectorID', 'Location'])
        for detector_id, location in data:
            writer.writerow([detector_id, location])

def main():
    directory1 = 'E:/xie/Sensor Files/5. dataset/full MICE imputed datasets/2018'
    directory2 = 'E:/xie/Sensor Files/5. dataset/full MICE imputed datasets/2019'
    output_file = 'E:/xie/common_sensor_data.csv'

    unique_data = get_unique_data(directory1, directory2)

    write_to_csv(unique_data, output_file)

    print("Unique data has been written to", output_file)

if __name__ == "__main__":
    main()


Unique data has been written to E:/xie/common_sensor_data.csv


In [ ]:
import csv

def read_csv(file_path):
    data = []
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row['DetectorID'])
    return data

def extract_sensor_parts(sensor_name, delimiter):
    """
    Extracts sensor parts from a sensor name.

    Args:
        sensor_name (str): The sensor name.
        delimiter (str): The delimiter used to separate parts.

    Returns:
        tuple: A tuple containing the sensor parts.
    """
    return tuple(sensor_name.split(delimiter))

def compare_sensor_names(your_csv_file, chloes_csv_file):
    sensor_names_file1 = read_csv(your_csv_file)
    sensor_names_file2 = read_csv(chloes_csv_file)

    # Extract sensor parts based on the delimiter used in each file
    sensor_parts_file1 = [extract_sensor_parts(name, '.') for name in sensor_names_file1]
    sensor_parts_file2 = [extract_sensor_parts(name, '_') for name in sensor_names_file2]

    common_sensor_names = set(sensor_parts_file1) & set(sensor_parts_file2)
    unique_sensor_names_file1 = set(sensor_parts_file1) - set(sensor_parts_file2)

    return common_sensor_names, unique_sensor_names_file1

def main():
    your_csv_file = 'E:/xie/common_sensor_data.csv'
    chloes_csv_file = 'E:/xie/Chloe - Reduced Detector Data - all_combined.csv'  # Provide the path to Chloe's CSV file

    common_sensor_names, unique_sensor_names_file1 = compare_sensor_names(your_csv_file, chloes_csv_file)
    
    print(f"Number of sensor names found in both files: {len(common_sensor_names)}")
    print("Common sensor names:")
    for sensor_parts in common_sensor_names:
        sensor_name = '.'.join(sensor_parts)  # Join sensor parts with '.'
        print(sensor_name)

    print(f"\nNumber of sensor names found only in your file: {len(unique_sensor_names_file1)}")
    print("Sensor names found only in your file:")
    for sensor_parts in unique_sensor_names_file1:
        sensor_name = '.'.join(sensor_parts)  # Join sensor parts with '.'
        print(sensor_name)

if __name__ == "__main__":
    main()


In [39]:
import csv

def read_csv(file_path):
    data = []
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

def extract_sensor_parts(sensor_name, delimiter):
    """
    Extracts sensor parts from a sensor name.

    Args:
        sensor_name (str): The sensor name.
        delimiter (str): The delimiter used to separate parts.

    Returns:
        tuple: A tuple containing the sensor parts.
    """
    return tuple(sensor_name.split(delimiter))

def compare_sensor_names(your_csv_file, chloes_csv_file):
    sensor_names_file1 = read_csv(your_csv_file)
    sensor_names_file2 = read_csv(chloes_csv_file)

    # Extract sensor parts based on the delimiter used in each file
    sensor_parts_file1 = {extract_sensor_parts(row['DetectorID'], '.') for row in sensor_names_file1}
    sensor_parts_file2 = {extract_sensor_parts(row['DetectorID'], '_') for row in sensor_names_file2}

    common_sensor_names = sensor_parts_file1 & sensor_parts_file2
    unique_sensor_names_file1 = sensor_parts_file1 - sensor_parts_file2

    # Find corresponding values for common sensor names
    common_sensor_values = []
    for sensor_name in common_sensor_names:
        for row in sensor_names_file2:
            if extract_sensor_parts(row['DetectorID'], '_') == sensor_name:
                common_sensor_values.append(row)
                break

    return common_sensor_values, unique_sensor_names_file1

def main():
    your_csv_file = 'E:/xie/common_sensor_data.csv'
    chloes_csv_file = 'E:/xie/Chloe - Reduced Detector Data - all_combined.csv'  # Provide the path to Chloe's CSV file

    common_sensor_values, unique_sensor_names_file1 = compare_sensor_names(your_csv_file, chloes_csv_file)

    # Save corresponding values for common sensor names to a new CSV file
    output_file = 'E:/xie/common_sensor_values.csv'
    fieldnames = ['DetectorID', 'Location', 'Latitude', 'Longitude']
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in common_sensor_values:
            writer.writerow({'DetectorID': row['DetectorID'], 'Location': row['Location'], 'Latitude': row['Latitude'], 'Longitude': row['Longitude']})

    print(f"Corresponding values for common sensor names have been saved to {output_file}")

if __name__ == "__main__":
    main()


Corresponding values for common sensor names have been saved to E:/xie/common_sensor_values.csv


In [40]:
def parse_location_description(description):
    """
    Parse a location description and extract relevant information.

    Args:
        description (str): The location description.

    Returns:
        dict: A dictionary containing parsed information.
    """
    parsed_info = {}

    # Split the description by 'between' to separate road names
    parts = description.split('between')

    # Extract road names
    if len(parts) == 2:
        road_names = [part.strip() for part in parts]
        parsed_info['road1'] = road_names[0]
        parsed_info['road2'] = road_names[1]

    # Extract direction and on-ramp/off-ramp information
    if 'OnRamp' in description:
        parsed_info['location_type'] = 'OnRamp'
        if 'SB' in description:
            parsed_info['direction'] = 'Southbound'
        elif 'NB' in description:
            parsed_info['direction'] = 'Northbound'
    elif 'OffRamp' in description:
        parsed_info['location_type'] = 'OffRamp'
        if 'SB' in description:
            parsed_info['direction'] = 'Southbound'
        elif 'NB' in description:
            parsed_info['direction'] = 'Northbound'

    return parsed_info

# Example usage:
description = "I-215 SB TownCenter OnRamp between Sahara Avenue and I-515 SB to Lake Mead Dr"
parsed_info = parse_location_description(description)
print(parsed_info)


{'road1': 'I-215 SB TownCenter OnRamp', 'road2': 'Sahara Avenue and I-515 SB to Lake Mead Dr', 'location_type': 'OnRamp', 'direction': 'Southbound'}


In [41]:
import re

def parse_location_description(description):
    """
    Parse a location description and extract relevant information.

    Args:
        description (str): The location description.

    Returns:
        dict: A dictionary containing parsed information.
    """
    parsed_info = {}

    # Define regular expressions to match different components
    highway_pattern = r'(I-\d{3}|US-\d{2}|CC-\d{3})'  # Matches highway names like I-15, US-95, CC-215
    direction_pattern = r'(NB|SB|EB|WB)'  # Matches directions like Northbound, Southbound, Eastbound, Westbound
    ramp_pattern = r'(OnRamp|OffRamp)'  # Matches on-ramp or off-ramp
    segment_pattern = r'between\s([\w\s]+?)\s+and\s+([\w\s]+)'  # Matches road segments between two points

    # Extract highway name
    highway_match = re.search(highway_pattern, description)
    if highway_match:
        parsed_info['highway'] = highway_match.group(0)

    # Extract direction
    direction_match = re.search(direction_pattern, description)
    if direction_match:
        parsed_info['direction'] = direction_match.group(0)

    # Extract ramp type
    ramp_match = re.search(ramp_pattern, description)
    if ramp_match:
        parsed_info['ramp_type'] = ramp_match.group(0)

    # Extract road segment
    segment_match = re.search(segment_pattern, description)
    if segment_match:
        parsed_info['segment'] = {
            'start': segment_match.group(1),
            'end': segment_match.group(2)
        }

    return parsed_info

# Example usage:
description = "I-215 SB TownCenter OnRamp between Sahara Avenue and I-515 SB to Lake Mead Dr"
parsed_info = parse_location_description(description)
print(parsed_info)


{'highway': 'I-215', 'direction': 'SB', 'ramp_type': 'OnRamp', 'segment': {'start': 'Sahara Avenue', 'end': 'I'}}


In [42]:
import re

def parse_location_description(description):
    """
    Parse a location description and extract relevant information.

    Args:
        description (str): The location description.

    Returns:
        dict: A dictionary containing parsed information.
    """
    parsed_info = {}

    # Define regular expressions to match different components
    highway_pattern = r'(I-\d{3}|US-\d{2}|CC-\d{3})'  # Matches highway names like I-15, US-95, CC-215
    direction_pattern = r'(NB|SB|EB|WB)'  # Matches directions like Northbound, Southbound, Eastbound, Westbound
    ramp_pattern = r'(OnRamp|OffRamp)'  # Matches on-ramp or off-ramp
    segment_pattern = r'between\s([\w\s]+?)\s+and\s+([\w\s]+)'  # Matches road segments between two points

    # Extract highway name
    highway_match = re.search(highway_pattern, description)
    if highway_match:
        parsed_info['highway'] = highway_match.group(0)

    # Extract direction
    direction_match = re.search(direction_pattern, description)
    if direction_match:
        parsed_info['direction'] = direction_match.group(0)

    # Extract ramp type if not in description
    if 'onramp' not in description.lower() and 'offramp' not in description.lower():
        ramp_match = re.search(ramp_pattern, description)
        if ramp_match:
            parsed_info['ramp_type'] = ramp_match.group(0)

    # Extract road segment
    segment_match = re.search(segment_pattern, description)
    if segment_match:
        parsed_info['segment'] = {
            'start': segment_match.group(1),
            'end': segment_match.group(2)
        }

    return parsed_info

# Example usage:
description = "I-215 SB TownCenter OnRamp between Sahara Avenue and I-515 SB to Lake Mead Dr"
parsed_info = parse_location_description(description)
print(parsed_info)


{'highway': 'I-215', 'direction': 'SB', 'segment': {'start': 'Sahara Avenue', 'end': 'I'}}


In [ ]:
# open a csv and save the first column as a separte csv
import csv  
 